# Challenge 8: Hidden Markov Models for genomic annotation

## BIOS 26211 winter 2023

## Your Name

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report

## Part 1: generating a string of observations from HMM

In this part you will write a function to generate a string of  observations according to a given HMM. 

The function requires the following:

* Inputs: vector of hidden states, transition matrix, vector of observations, observation matrix, vector of initial probabilities and the length of the output string. 
* Do:  Assign the hidden state of the first character from the initial distribution
    * Generate each character in the string in two steps:
        * pick a hidden state according to the transition probability, given the previous hidden state
        * pick an observed state according the observation probability for the selected hidden state
* Output: string of hidden states and string of observations 

The following script defines the set of states (non-CpG 'Non' and CpG 'CpG') and observations (nucleotides), the transition probabilities between the hidden states and the probabilities of observing each nucleotide for the two hidden states. 

The initial state can be sampled from the given probability distribution using the function `numpy.random.choice` as shown below:

In [3]:
states = np.array(['Non', 'CpG']) # define the names of the states (Non and CpG regions)
trans_mat = np.array([[0.7, 0.1], [0.3, 0.9]]) # define a 2 x 2 transition matrix between the hidden states
nucleotides = np.array(['A', 'T', 'G', 'C'])  # define the alphabet of nucleotides
Non_obs_probs = np.array([[0.47, 0.47, 0.03, 0.03]])  # the probabilities of observations for the AT-rich state
CpG_obs_probs = np.array([[0.03, 0.03, 0.47, 0.47]])# the probabilities of observations for the GC-rich state
obs_mat = np.append(Non_obs_probs, CpG_obs_probs, axis = 0) # create a 2 x 4 matrix of observation probabilities
initial = np.random.choice(states, 1, p=[0.7, 0.3])
print(obs_mat)

[[0.47 0.47 0.03 0.03]
 [0.03 0.03 0.47 0.47]]


### 1.1 
Write a function that takes in the 6 inputs listed above, generates a string of hidden states and a string of observations and returns them both. Test this function by generating a random string of 10 letters with the transition and observation matrix given above and an initial probability distribution of your choice. Repeat the generation and report how the resulting strings are different.

Your answer

### 1.2 
Change the transition matrix so that the transition probabilites are 1 and 0 (either that the states stay the same or they always change) and change the observation matrix so that only A and T are observed in the Non states and only C and G are observed in the CpG state and run the function again to generate a string of 10 letters with an initial probability distribution of your choice. Repeat the generation and report how the resulting strings are different. 

Your answer here

## Part 2: implementing and testing the Viterbi algorithm

In this part you will implement the Viterbi algorithm, which calculates the most likely string of hidden states based on a given HMM and a string of observations. A pseudocode for the algorithm is available here: https://en.wikipedia.org/wiki/Viterbi_algorithm#Pseudocode 

* Inputs: 1) list of hidden states, 2) list of observed states, 3) transition matrix between hidden states, 4) observation (emission) probability matrix, 5) the vector of initial  probabilities and 6) string of observed states. 
* initialize two data structures: matrix of Viterbi scores and traceback matrix
* loop over all the observations in the string
    * loop over all the hidden states
        * compute a matrix of partial Viterbi scores for each element in row i (corresponding to hidden state i), column j (corresponding to observation number j) called $v_i(j)$, by assigning the maximum of probabilities that lead to that particular cell, chosen from $a_{ik}v_k(j-1)$ for all states k, where $a_{ik}$ represent the transition probabilities from states k to state i, then multiply it by the emission probability of the character by that state
        * record the origin of the highest probability for this element $[i,j]$ of the traceback matrix
* pick the maximum score in the last column, traceback from that cell to produce the most likely state string
* Output: the string of most likely hidden states. 

Comments: 

1. the pseudocode uses poor variable names, particularly as it uses T_1 for the matrix of probability scores, T_2 for the traceback matrix, and T for the length of the sequence. Please choose better variable names! 
2. the subscripts in the pseudocode mean indices
3. for better accuracy *add up logs of probabilities instead of multiplying them*
4. the output string should be an array of three-letter character strings, which can be initialized as shown in example below

### 2.1 

Test the function by using the same inputs you used in question 1.2.  Generate a new string of 30 observations and then run the Viterbi algorithm on that string to reconstruct the string of hidden states. Compare the string of hidden states from the Viterbi algorithm with the string of hidden states generated by your function from part 1 using the function `classification_report`; because the transition matrix is not actually random, you should see perfect accuracy (but may see warnings due to division by 0, don't worry about them).

In [43]:
string = np.repeat('   ', 5) # initialize string

### 2.2 
Change the transition and observation matrix back to the same  parameter values provided in the code for part 1.1 of the assignment. Generate a new string of 30 observations, and then run the Viterbi algorithm to reconstruct the string of hidden states. Compare the string of hidden states from the Viterbi algorithm with the actual string of hidden states and report the fraction of correct hidden states. Repeat this experiment 20 times (use a loop!) and report on the quality of the hidden state reconstruction produced by the Viterbi algorithm  for this HMM using the function `classification_report`. 

Your answer here

### 2.3 
Change the matrix of observation probabilities so that for state 'N' the emission probabilities of 'A' and 'T' are both 0.4, and the emission probabilities of 'G' and 'C' are both 0.1; for state 'C'  the emission probabilities of 'G' and 'C' are both 0.4, and the emission probabilities of 'A' and 'T' are both 0.1. Generate a new string of 30 observations, and then run the Viterbi algorithm to reconstruct the string of hidden states and report the fraction of correct hidden states.  Repeat this experiment 20 times (use a loop!) and report on the quality of the hidden state reconstruction produced by the Viterbi algorithm for this HMM using the function `classification_report`. Explain why the prediction quality is different in the two cases.

Your answer here

## Part 3: Using real data

The following sript reads in the sequence from a fragment of human X chromosome; information from https://www.ncbi.nlm.nih.gov/nuccore/L44140 . The sequence has been annotated with different functions, among them are 16 CpG islands. The sequence is read from text file 'L44140.txt' into the character string `chrom_seq`, and arrays `start_positions` and `end_position` contain the locations of the start and the end nucleotides of the CpG islands.

In [40]:
chrom_seq, blah = np.loadtxt("L44140.txt", dtype=str, delimiter = None)
print("Length of the sequence:", len(chrom_seq))

start_positions = np.array([3095, 11638, 40983, 48446, 59461, 67900, 81836, 98783,
                  106826, 114316, 128187, 148990, 156388, 160697, 186412, 216617]) 
end_positions = np.array([3426, 13564, 42150, 50350, 61404, 69462, 82633, 99468,
                108158, 114957, 129236, 149796, 157495, 161402, 186922, 217876])


Length of the sequence: 219447


### 3.1 calculating HMM parameters

Calculate the parameters for a two state HMM, like the example given above.

1. Calculate the transition probabilities between Non-CpG and CpG states by dividing the number of transitions between states by the total length of the CpG and non-CpG regions, respectively.

2. Calculate the emission probabilities for the Non-CpG and CpG states; you'll find useful the count function that will count the occurences of a character in a string (see example below), and use a loop to go through the start and end positions.

3. Assign the string of hidden states 'Non' and 'CpG', using the start and end positions to assign these values to Non-CpG and CpG islands.


Print out the transition and emission matrices and compare them to the toy examples from parts 1 and 2.

In [44]:
print("Number of As:", chrom_seq.count('A'))


Number of As: 46881


### 3.2 Use Viterbi to predict CpG islands

Use the parameters you calculated above and the entire sequence to predict where the CpG islands are. (Yes this is technically using the training set for testing, but there's not much danger of overfitting; you can try splitting the data if you want to be more careful.)

Start with initial state 'Non', and feed the entire string `chrom_seq` to your Viterbi function to generate the most likely sequence of hidden states. Then use the classification_report function to quantify the prediction quality. 

Your answer here